# Counterfactual Regret Minimization with Kuhn Poker

First, we import NumPy and define constants for the number of cards and number of possible actions. In our algorithm each player only has two possible actions: check and bet. This simplifies our code. This is possible because a fold can be treated as a check, and a call can be treated as a bet without loss of generality.

In [1]:
import numpy as np

N_ACTIONS = 2 # 0 = check, 1 = bet
N_CARDS = 3 # 0 = jack, 1 = queen, 2 = king
  

In Vanilla CFR each information set is visited multiple times each iteration. InformationSet stores self.regret_sum, self.strategy_sum, self.strategy, self.reach_pr, and self.reach_pr_sum. self.regret_sum, self.strategy_sum, and self.strategy are arrays indexed by an action. In our algorithm check is index 0 and bet is index 1. self.regret_sum is the sum of counterfactual regrets for each action over all visits. self.strategy_sum is the sum of each visit’s strategy multiplied by the information set player’s reach probability. self.regret_sum is used to calculate the next strategy to try. self.strategy_sum and self.reach_pr_sum is used to calculate the average strategy. self.strategy is the strategy for the current iteration and self.reach_pr accumulates the probability of reaching an information set. As more iterations are performed, average regret will minimize and the average strategy will converge to a Nash equilibrium.

In [2]:
class InformationSet():
    """
    Classe che rappresenta un information set.
    """
    def __init__(self, key):
        # key = history + carta del giocatore
        self.key = key
        # array contentente la somma dei counterfactual regrets per ogni azione in tutte le visite dell'information set. 
        # Check = index 0, Bet = index 1
        self.regret_sum = np.zeros(N_ACTIONS) 
        # array contentente la somma della strategie di ogni visita moltiplicata per la reach probability del player corrente
        self.strategy_sum = np.zeros(N_ACTIONS) 
        self.strategy = np.repeat(1/N_ACTIONS, N_ACTIONS) # array contentente la strategia corrente (inizializzata a uniforme)
        self.reach_pr = 0 # reach probability del player corrente
        self.reach_pr_sum = 0

    def next_strategy(self):
        """
        Aggiorna la strategia corrente e la somma delle strategie.
        """
        self.strategy_sum += self.reach_pr * self.strategy
        self.strategy = self.calc_strategy()
        self.reach_pr_sum += self.reach_pr
        self.reach_pr = 0

    def calc_strategy(self):
        """
        Calcola la strategia corrente a partire dai counterfactual regrets.
        Sceglie una strategia proporzionale ai agli elementi positivi del regret_sum facendo il Regret Matching.
        """
        
        strategy = np.where(self.regret_sum > 0, self.regret_sum, 0) # prende solo i regret positivi

        total = sum(strategy) # somma dei regret positivi
        if total > 0: 
            strategy = strategy / total # normalizza i regret positivi
        else:
            n = N_ACTIONS
            strategy = np.repeat(1/n, n) # se non ci sono regret positivi, sceglie una strategia uniforme

        return strategy

    def get_average_strategy(self):
        """
        Calculate average strategy over all iterations. This is the
        Nash equilibrium strategy.
        """
        strategy = self.strategy_sum / self.reach_pr_sum

        # Purify to remove actions that are likely a mistake
        strategy = np.where(strategy < 0.001, 0, strategy)

        # Re-normalize
        total = sum(strategy)
        strategy /= total

        return strategy


    def __str__(self):
        strategies = ['{:03.2f}'.format(x)
                      for x in self.get_average_strategy()]
        return '{} {}'.format(self.key.ljust(6), strategies)

The function recursively performs a depth-first traverse across the game tree. In Vanilla CFR we traverse the entire game tree every iteration. cfr() performs different tasks depending on if the node is a chance node, a terminal node, or a decision node. It takes seven parameters.

- i_map is the hash map of information sets.
history is a string that represents our current location in the game tree. Each character represents an action we have taken and an edge we have followed.
- ‘r’ is a random chance event.
- ‘c’ is a check action.
- ‘b’ is a bet action.
- card_1 is the private card of player one.
- card_2 is the private card of player two.
- pr_1 is player one’s contribution to the reach probability—the probability that we’ve reached history.
- pr_2 is player two’s contribution to the reach probability.
- pr_c is the contribution of chance events to the reach probability.

In [3]:

def cfr(i_map, history="", card_1=-1, card_2=-1, pr_1=1, pr_2=1, pr_c=1):
    """
    Counterfactual regret minimization algorithm.

    Parameteri
    ----------
    i_map: dizionario degli information set
    history : [{'r', 'c', 'b'}], stringa che rappresenta il path preso nel game tree
        'r': random chance action
        'c': check action
        'b': bet action
    card_1 : carta del player 1
    card_2 : carta del player 2
    pr_1 : Probabilità che il player 1 arrivi a history
    pr_2 : Probabilità che il player 2 arrivi a history
    pr_c: Contributo di probabilità del chance node per raggiungere history

    """
    
    # Se l'history è un chance node, allora ritorna il valore atteso
    if is_chance_node(history):
        return chance_util(i_map)
    
    # Se l'history è un terminal node, allora ritorna la terminal utility di questa combinazione di carte
    if is_terminal(history):
        return terminal_util(history, card_1, card_2)

    # --------------Se arrivo qui, allora l'history è un decision node----------------
    
    # Calcolo il numero di azioni che sono state prese fino ad ora
    n = len(history)    
    # Se n è pari, allora è il turno del player 1, altrimenti è il turno del player 2
    is_player_1 = n % 2 == 0
    # Deduco l'infromation set della history del player corrente
    info_set = get_info_set(i_map, card_1 if is_player_1 else card_2, history)
    # Deduco la strategia corrente del player corrente
    strategy = info_set.strategy
    # Aggiungo la reach probability del player corrente all'information set
    if is_player_1:
        info_set.reach_pr += pr_1
    else:
        info_set.reach_pr += pr_2

    # Counterfactual utility per azione
    action_utils = np.zeros(N_ACTIONS)

    # Chiamo ricorsivamente cfr per ogni azione possibile  
    for i, action in enumerate(["c", "b"]):
        next_history = history + action
        if is_player_1:
            action_utils[i] = -1 * cfr(i_map= i_map, history= next_history,
                                       card_1= card_1, card_2= card_2,
                                       pr_1= pr_1 * strategy[i], pr_2= pr_2, pr_c= pr_c) 
        else:
            action_utils[i] = -1 * cfr(i_map = i_map, history= next_history,
                                       card_1= card_1,card_2= card_2,
                                       pr_1= pr_1,pr_2= pr_2 * strategy[i],pr_c= pr_c)

    # Calcolo i counterfactual regrets
    util = sum(action_utils * strategy) # somma delle utility delle azioni pesate con la strategia corrente
    regrets = action_utils - util # calcolo i counterfactual regrets
    # Aggiorno i counterfactual regrets dell'information set
    if is_player_1:
        info_set.regret_sum += pr_2 * pr_c * regrets
    else:
        info_set.regret_sum += pr_1 * pr_c * regrets
    
    return util

In [4]:
def card_str(card):
    """
    Ritorna una string representation delle carte.
    """
    if card == 0:
        return "J"
    elif card == 1:
        return "Q"
    return "K"


def get_info_set(i_map, card, history):
    """
    Ritorna l'information set associato alla carta e alla history.
    Se non esiste, allora lo crea e lo aggiunge al dizionario.
    """
    # La chiave è composta da una stringa che rappresenta la carta e la history
    key = card_str(card) + " " + history
    info_set = None
    
    # Se la chiave non è presente nel dizionario, allora creo un nuovo information set
    if key not in i_map:
        # Creo un'istanza della classe InformationSet
        info_set = InformationSet(key)
        # Aggiungo il nuovo information set al dizionario
        i_map[key] = info_set
        return info_set
    
    # Altrimenti ritorno l'information set associato alla chiave
    return i_map[key]

is_chance_node() determines if we are at a chance node by checking for an empty history. 
This works because in Kuhn Poker chance events only occur at the start of the game. If is_chance_node() returns true then chance_util() enumerates all possible combinations of chance nodes. There are six total possible combinations. For each possibility, we recursively call cfr(). The next history is "rr" to represent two random chance actions. Neither player has taken any actions, so their reach probabilities are 1. Each chance event has a uniformly random probability of occurring of 1/n_possibilities. The expected value over all possibilities is just the sum of the utility of each possibility divided by the number of possibilities.

In [5]:
def is_chance_node(history):
    return history == "" # return True if history == "" else False

# se sono in un chance node
def chance_util(i_map):
    expected_value = 0
    n_possibilities = 6
    # 6 possibili combinazioni di carte
    for i in range(N_CARDS):
        for j in range(N_CARDS):
            if i != j:
                # ottengo l'ev di ogni combinazione
                expected_value += cfr(i_map= i_map,
                                      history= "rr",
                                      card_1= i,
                                      card_2= j,
                                      pr_1= 1, # probabilità che il giocatore 1 arrivi a quel nodo è 1 perchè ancora non ha giocato
                                      pr_2= 1, # probabilità che il giocatore 2 arrivi a quel nodo è 1 perchè ancora non ha giocato
                                      pr_c= 1/n_possibilities) # probabilità che il chance node arrivi a quel nodo è 1/6 perchè è uan distribuzione uniforme
    return expected_value/n_possibilities # ritorno la media degli ev

In [6]:
def is_terminal(history):
    """
    Ritorna True se l`history` è una terminal history.
    """    
    # 5 possibili terminal history
    possibilities = {"rrcc": True, # showdown con check e check
                     "rrbb": True, # showdown con bet e bet(call)                  
                     "rrcbb": True, # showdown con check, bet e bet(call)
                     "rrbc": True, # non showdown con bet e check(fold)
                     "rrcbc": True, # non showdown con check, bet e check(fold)
                     }
    return history in possibilities


def terminal_util(history, card_1, card_2):
    """
    Ritorna l'utility di una terminal history per il player corrente. 
    Siccome i players is alternano ad ogni turno, se il numero di azioni è pari allora
    il player corrente è il player 1 altrimenti è il player 2 
    """
    n = len(history) # numero di azioni fatte
    # se il numero di azioni è pari allora è il player 1 altrimenti è il player 2
    card_player = card_1 if n % 2 == 0 else card_2 
    card_opponent = card_2 if n % 2 == 0 else card_1

    # No showdown
    if history == "rrcbc" or history == "rrbc":
        # Opponent ha foldato, il player corrente vince 1
        return 1
    # Showdown senza bets
    elif history == "rrcc":
        # Chi ha la carta più alta vince 1
        return 1 if card_player > card_opponent else -1

    # Showdown con bet e call
    if history == "rrcbb" or history == "rrbb":
        # Chi ha la carta più alta vince 2
        return 2 if card_player > card_opponent else -2

In [7]:
def display_results(ev, i_map):
    print('player 1 expected value: {}'.format(ev))
    print('player 2 expected value: {}'.format(-1 * ev))

    print()
    print('player 1 strategies:')
    print(f"History  Check    Bet")
    sorted_items = sorted(i_map.items(), key=lambda x: x[0])
    for _, v in filter(lambda x: len(x[0]) % 2 == 0, sorted_items):
        print(v)
    print()
    print('player 2 strategies:')
    print(f"History  Check    Bet")
    for _, v in filter(lambda x: len(x[0]) % 2 == 1, sorted_items):
        print(v)
        

Each iteration of cfr() returns the expected game value for that iteration. We are able to approximate the true game value by averaging over all iterations. The true game value is the expected amount that a player will win while following the average strategy.

In [8]:

i_map = {}   # dizionario degli information set
n_iterations = 10000 # numero di iterazioni
expected_game_value = 0

for _ in range(n_iterations): # run CFR algorithm n_iterations times
    expected_game_value += cfr(i_map) # update game value
    for _, v in i_map.items():
        v.next_strategy() # update strategy

expected_game_value /= n_iterations

display_results(expected_game_value, i_map) 

player 1 expected value: -0.05666979368427769
player 2 expected value: 0.05666979368427769

player 1 strategies:
History  Check    Bet
J rr   ['0.79', '0.21']
J rrcb ['1.00', '0.00']
K rr   ['0.39', '0.61']
K rrcb ['0.00', '1.00']
Q rr   ['1.00', '0.00']
Q rrcb ['0.45', '0.55']

player 2 strategies:
History  Check    Bet
J rrb  ['1.00', '0.00']
J rrc  ['0.67', '0.33']
K rrb  ['0.00', '1.00']
K rrc  ['0.00', '1.00']
Q rrb  ['0.66', '0.34']
Q rrc  ['1.00', '0.00']


If you run the program for 100,000 iterations it will produce a slightly better approximation. Some CFR implementations have run for up to 6 million core hours [6]. As you approach an infinite number of iterations the overall average regret will approach zero and the average strategy will approach a Nash equilibrium.

The strategies indicate that if player one has a J her first move should be to check 79% of the time and bet 21% of the time. If player two has a K and player one checked, he should bet 100% of the time. This is just one Nash equilibrium. Kuhn Poker has infinitely many. Vanilla CFR is a deterministic algorithm, but many variants are non-deterministic. A non-deterministic CFR may find a different Nash equilibrium.